In [1]:
from datasets import load_dataset
import pandas as pd
import re
from bs4 import BeautifulSoup
from langdetect import detect


/home/daisy_cherono/myenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("derekiya/swahili_news")

In [3]:
print(dataset['train'])

Dataset({
    features: ['text', 'label'],
    num_rows: 24210
})


In [4]:
print(dataset['test'])

Dataset({
    features: ['text', 'label'],
    num_rows: 7338
})


In [5]:
df_train = pd.DataFrame(dataset['train'])

In [11]:
df_test = pd.DataFrame(dataset['test'])

In [41]:
print(df_train.tail())

                                                    text  label
24205   Alitoa pongezi hizo alipozindua rasmi hatua y...      0
24206   Na NORA DAMIAN-DAR ES SALAAM  TEKLA (si jina ...      1
24207   Mkuu wa Mkoa wa Njombe, Dk Rehema Nchimbi wak...      0
24208   MABINGWA wa Ligi Kuu Soka Tanzania Bara, Simb...      2
24209   WIKI iliyopita, nilianza makala haya yanayole...      1


In [42]:
print(df_test.tail())

                                                   text  label
7333   Kamati hiyo ilibainisha kuwa moja ya mapunguf...      0
7334  ARODIA PETER-DODOMA HOSPITALI ya Rufaa ya Benj...      1
7335   WAKATI mazoezi ya timu ya taifa ya Tanzania (...      2
7336  \n\tNa Suleiman Rashid Omar-Pemba\n \n\n \n\tW...      1
7337   BAO pekee lililofungwa na mshambuliaji wa Yan...      2


In [13]:
#def normalize_text(text):
   # return text.lower()

def remove_punctuation_and_normalize_whitespace(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def remove_html_tags_and_special_characters(text):
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    return text

def is_swahili(text):
    try:
        return detect(text) == 'sw'
    except:
        return False

def preprocess_text(text):
    #text = normalize_text(text)
    text = str(text)
    text = remove_punctuation_and_normalize_whitespace(text)
    text = remove_html_tags_and_special_characters(text)
    return text


In [32]:
cleaned_texts = []
labels = []

for text, label in zip(df_train['text'], df_train['label']):
    cleaned_text = preprocess_text(text)
    
    if is_swahili(cleaned_text):
        cleaned_texts.append(cleaned_text)
        labels.append(label)

In [33]:
cleaned_train = pd.DataFrame({'text': cleaned_texts, 'label': labels})

In [40]:
print(cleaned_train.tail())

                                                    text  label
24170  Alitoa pongezi hizo alipozindua rasmi hatua ya...      0
24171  Na NORA DAMIANDAR ES SALAAM TEKLA si jina lake...      1
24172  Mkuu wa Mkoa wa Njombe Dk Rehema Nchimbi wakat...      0
24173  MABINGWA wa Ligi Kuu Soka Tanzania Bara Simba ...      2
24174  WIKI iliyopita nilianza makala haya yanayoleng...      1


In [36]:
cleaned_text_test = []
labels_test = []

for text, label in zip(df_test['text'], df_test['label']):
    cleaned_text = preprocess_text(text)
    
    if is_swahili(cleaned_text):
        cleaned_text_test.append(cleaned_text)
        labels_test.append(label)

In [37]:
cleaned_test = pd.DataFrame({'text': cleaned_text_test, 'label': labels_test})

In [38]:
print(cleaned_test.tail())

                                                   text  label
7330  Kamati hiyo ilibainisha kuwa moja ya mapungufu...      0
7331  ARODIA PETERDODOMA HOSPITALI ya Rufaa ya Benja...      1
7332  WAKATI mazoezi ya timu ya taifa ya Tanzania Ta...      2
7333  Na Suleiman Rashid OmarPemba WIZARA ya Afya na...      1
7334  BAO pekee lililofungwa na mshambuliaji wa Yang...      2
